In [8]:
# Install required packages in Colab (if not already installed)
!pip install kagglehub pillow numpy tensorflow

import os
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import kagglehub
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image

# Print TensorFlow version for debugging
print("TensorFlow version:", tf.__version__)

# -------------------------------
# Custom Functions for CBAM (Registered for Serialization)
# -------------------------------
@tf.keras.utils.register_keras_serializable()
def spatial_avg_pool(x):
    return tf.reduce_mean(x, axis=-1, keepdims=True)

@tf.keras.utils.register_keras_serializable()
def spatial_max_pool(x):
    return tf.reduce_max(x, axis=-1, keepdims=True)

# -------------------------------
# 1. Download and Prepare Dataset via Kagglehub
# -------------------------------
dataset_path = kagglehub.dataset_download("markyousri/marine-life-classification-dataset")
print("Path to dataset files:", dataset_path)

root_dir = os.path.join(dataset_path, "Dataset")
if not os.path.exists(root_dir):
    raise FileNotFoundError("Expected 'Dataset' directory not found. Check dataset structure.")

classification_dir = os.path.join(root_dir, "classification")
if not os.path.exists(classification_dir):
    raise FileNotFoundError("Could not find classification dataset in expected paths. Check dataset structure.")

train_dir = os.path.join(classification_dir, "train")
val_dir = os.path.join(classification_dir, "val")

# -------------------------------
# 2. Data Loading and Augmentation
# -------------------------------
img_size = (300, 300)
batch_size = 32

# Load datasets and capture class_names before transformations
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)
class_names = train_ds.class_names  # Store class names separately

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

num_classes = len(class_names)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
])

# -------------------------------
# 3. CBAM Attention Block
# -------------------------------
def cbam_block(input_feature, ratio=8):
    channel = input_feature.shape[-1]

    shared_dense_one = layers.Dense(channel // ratio, activation='relu',
                                   kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
    shared_dense_two = layers.Dense(channel, kernel_initializer='he_normal',
                                   use_bias=True, bias_initializer='zeros')

    avg_pool = layers.GlobalAveragePooling2D()(input_feature)
    avg_pool = layers.Reshape((1, 1, channel))(avg_pool)
    avg_out = shared_dense_two(shared_dense_one(avg_pool))

    max_pool = layers.GlobalMaxPooling2D()(input_feature)
    max_pool = layers.Reshape((1, 1, channel))(max_pool)
    max_out = shared_dense_two(shared_dense_one(max_pool))

    channel_attention = layers.Add()([avg_out, max_out])
    channel_attention = layers.Activation('sigmoid')(channel_attention)
    channel_refined = layers.Multiply()([input_feature, channel_attention])

    avg_pool_spatial = layers.Lambda(spatial_avg_pool,
                                   output_shape=(input_feature.shape[1], input_feature.shape[2], 1))(channel_refined)
    max_pool_spatial = layers.Lambda(spatial_max_pool,
                                   output_shape=(input_feature.shape[1], input_feature.shape[2], 1))(channel_refined)

    concat = layers.Concatenate(axis=-1)([avg_pool_spatial, max_pool_spatial])
    spatial_attention = layers.Conv2D(filters=1, kernel_size=7, padding='same', activation='sigmoid',
                                     kernel_initializer='he_normal', use_bias=False)(concat)
    refined_feature = layers.Multiply()([channel_refined, spatial_attention])
    return refined_feature

# -------------------------------
# 4. Build the Model
# -------------------------------
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=img_size + (3,))
base_model.trainable = False  # Freeze base model for transfer learning

inputs = layers.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
preprocessed = tf.keras.applications.efficientnet.preprocess_input(x)
x = base_model(preprocessed, training=False)
x = cbam_block(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs, outputs)

# -------------------------------
# 5. Compile and Train the Model
# -------------------------------
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stop, lr_scheduler]
)

# -------------------------------
# 6. Save the Model
# -------------------------------
model.save("marine_life_species_classifier.keras")
print("Model saved as marine_life_species_classifier.keras")

# -------------------------------
# 7. Load and Test Model
# -------------------------------
model = tf.keras.models.load_model("marine_life_species_classifier.keras")

def predict_image(img_path):
    try:
        img = Image.open(img_path)
        img = img.convert("RGB")
        img = img.resize(img_size)
        img_array = image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)
        img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)

        predictions = model.predict(img_array)
        class_idx = np.argmax(predictions)
        class_name = class_names[class_idx]  # Use stored class_names
        confidence = predictions[0][class_idx]

        result = f"Predicted class: {class_name} with confidence: {confidence:.2f}"
        print(result)
        return class_name, confidence
    except (FileNotFoundError, Image.UnidentifiedImageError, ValueError) as e:
        print(f"Error processing image: {e}")
        return None, None


TensorFlow version: 2.18.0
Path to dataset files: /kaggle/input/marine-life-classification-dataset
Found 1162 files belonging to 5 classes.
Found 308 files belonging to 5 classes.


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 300, 300, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_2 (Sequential) │ (None, 300, 300, 3)    │              0 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ efficientnetb3            │ (None, 10, 10, 1536)   │     10,783,535 │ sequential_2[0][0]     │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 1536)           │              0 │ efficientnetb3[0][0]   │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling2d_2    │ (None, 1536)           │              0 │ efficientnetb3[0][0]   │
│ (GlobalMaxPooling2D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_4 (Reshape)       │ (None, 1, 1, 1536)     │              0 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_5 (Reshape)       │ (None, 1, 1, 1536)     │              0 │ global_max_pooling2d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 1, 1, 192)      │        295,104 │ reshape_4[0][0],       │
│                           │                        │                │ reshape_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 1, 1, 1536)     │        296,448 │ dense_8[0][0],         │
│                           │                        │                │ dense_8[1][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 1, 1, 1536)     │              0 │ dense_9[0][0],         │
│                           │                        │                │ dense_9[1][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 1, 1, 1536)     │              0 │ add_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply_4 (Multiply)     │ (None, 10, 10, 1536)   │              0 │ efficientnetb3[0][0],  │
│                           │                        │                │ activation_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_4 (Lambda)         │ (None, 10, 10, 1)      │              0 │ multiply_4[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_5 (Lambda)         │ (None, 10, 10, 1)      │              0 │ multiply_4[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 10, 10, 2)      │              0 │ lambda_4[0][0],        │
│ (Concatenate)             │                        │                │ lambda_5[0][0]         │
├──────────────────────

 Total params: 11,578,710 (44.17 MB)

 Trainable params: 792,103 (3.02 MB)

 Non-trainable params: 10,786,607 (41.15 MB)

Epoch 1/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 39s 450ms/step - accuracy: 0.7242 - loss: 3.0420 - val_accuracy: 0.9675 - val_loss: 2.2375 - learning_rate: 0.0010
Epoch 2/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 321ms/step - accuracy: 0.9407 - loss: 1.9088 - val_accuracy: 0.9708 - val_loss: 1.6374 - learning_rate: 0.0010
Epoch 3/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 306ms/step - accuracy: 0.9542 - loss: 1.4163 - val_accuracy: 0.9740 - val_loss: 1.2249 - learning_rate: 0.0010
Epoch 4/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 11s 308ms/step - accuracy: 0.9647 - loss: 1.0915 - val_accuracy: 0.9740 - val_loss: 0.9543 - learning_rate: 0.0010
Epoch 5/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 313ms/step - accuracy: 0.9674 - loss: 0.8355 - val_accuracy: 0.9740 - val_loss: 0.7350 - learning_rate: 0.0010
Epoch 6/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 321ms/step - accuracy: 0.9814 - loss: 0.6563 - val_accuracy: 0.9740 - val_loss: 0.5935 - learning_rate: 0.0010
Epoch 7/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 320ms/step - accuracy: 0.9752 - loss: 0.

In [11]:
# Example usage: Upload an image to Colab and provide its path
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
    predict_image(filename)

Saving whale.jpg to whale (1).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predicted class: whale with confidence: 0.98
